# Descriptions

The notebook demonstrates how to use `ImClient`.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Imports

In [2]:
import logging

import pandas as pd

import core.config as cconfig
import core.finance as cofinanc
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import im_v2.ccxt.data.client as icdcl
import im_v2.common.data.client as icdc
import im_v2.common.db.db_utils as imvcddbut
import im_v2.common.universe as ivcu

/app/amp/core/statistics/random_samples.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
log_level = logging.INFO
hdbg.init_logger(verbosity=log_level)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-6a09ab9b-0e1e-4e98-9edf-7cd1038fa7b0.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.10.0' != container_version='1.12.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask5923_Add_ImClient_gallery_notebook'
  hash='b2d8936b7'
  # Last commits:
    *   b2d8936b7 dan      Merge branch 'CmTask5923_Add_ImClient_gallery_notebook' of github.com:cryptokaizen/cmamp into CmTask5923_Add_ImClient_gallery_notebook (25 minutes ago) Thu Dec 14 15:06:47 2023  (HEAD -> CmTask5923_Add_ImClient_gallery_notebook, origin/CmTask5923_Add_ImClient_gallery_notebook)
    |\  
    | *   64b0bcfc9 Dan      Merg

# Config

In [4]:
config = {
    "universe": {
        "vendor": "CCXT",
        "mode": "trade",
        "version": "v7.4",
        "as_full_symbol": True,
    },
    "start_timestamp": pd.Timestamp("2023-09-11T00:00:00", tz="UTC"),
    "end_timestamp": pd.Timestamp("2023-09-11T04:00:00", tz="UTC"),
    "columns": None,
    "filter_data_mode": "assert",
    "ohlcv_parquet_config": {
        "vendor": "ccxt",
        "universe_version": "v7.4",
        "root_dir": "s3://cryptokaizen-data/v3",
        "partition_mode": "by_year_month",
        "dataset": "ohlcv",
        "contract_type": "futures",
        "data_snapshot": "",
        "aws_profile": "ck",
        "resample_1min": False,
        "version": "v1_0_0",
        "tag": "downloaded_1min",
    },
    "bid_ask_parquet_config": {
        "vendor": "ccxt",
        "universe_version": "v7.4",
        "root_dir": "s3://cryptokaizen-data-test/v3",
        "partition_mode": "by_year_month",
        "dataset": "bid_ask",
        "contract_type": "futures",
        "data_snapshot": "",
        "version": "v1_0_0",
        "tag": "resampled_1min",
        "aws_profile": "ck",
    },
    "ohlcv_db_config": {
        "universe_version": "infer_from_data",
        "db_connection": imvcddbut.DbConnectionManager.get_connection("preprod"),
        "table_name": "ccxt_ohlcv_futures",
        "resample_1min": False,
    },
}
config = cconfig.Config().from_dict(config)
print(config)

INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Unable to fetch DB credentials from env file: 
	
################################################################################
* Failed assertion *
'preprod' in '['local', 'dev', 'prod']'
################################################################################

	Attempting AWS SecretsManager method.
INFO  Fetching secret: preprod.im_data_db
INFO  Created preprod DB connection: 
 None
universe: 
  vendor: CCXT
  mode: trade
  version: v7.4
  as_full_symbol: True
start_timestamp: 2023-09-11 00:00:00+00:00
end_timestamp: 2023-09-11 04:00:00+00:00
columns: None
filter_data_mode: assert
ohlcv_parquet_config: 
  vendor: ccxt
  universe_version: v7.4
  root_dir: s3://cryptokaizen-data/v3
  partition_mode: by_year_month
  dataset: ohlcv
  contract_type: futures
  data_snapshot: 
  aws_profile: ck
  resample_1min: False
  version: v1_0_0
  tag: downloaded_1min
bid_ask

# CCXT Binance 1-minute futures data

In [5]:
full_symbols = ivcu.get_vendor_universe(**config["universe"])
_LOG.info("Full symbols number=%s", len(full_symbols))
full_symbols[:5]

INFO  Full symbols number=24


['binance::APE_USDT',
 'binance::AVAX_USDT',
 'binance::AXS_USDT',
 'binance::BAKE_USDT',
 'binance::BNB_USDT']

## OHLCV Parquet

In [6]:
ohlcv_im_client = icdc.HistoricalPqByCurrencyPairTileClient(
    **config["ohlcv_parquet_config"]
)

In [7]:
ohlcv_data = ohlcv_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
ohlcv_data.head()

,full_symbol,open,high,low,close,volume,knowledge_timestamp
timestamp,,,,,,,
2023-09-11 00:00:00+00:00,binance::APE_USDT,1.2260,1.2260,1.2220,1.225,215214.00,2023-09-12 00:18:08.017865+00:00
2023-09-11 00:00:00+00:00,binance::AVAX_USDT,9.4570,9.4590,9.4470,9.455,24887.00,2023-09-12 00:18:05.672893+00:00
2023-09-11 00:00:00+00:00,binance::AXS_USDT,4.3890,4.3920,4.3840,4.391,12163.00,2023-09-12 00:18:23.505853+00:00
2023-09-11 00:00:00+00:00,binance::BAKE_USDT,0.0949,0.0951,0.0949,0.095,428825.00,2023-09-12 00:18:18.777530+00:00
2023-09-11 00:00:00+00:00,binance::BNB_USDT,212.2800,212.4000,212.2700,212.300,840.62,2023-09-12 00:18:06.849434+00:00


## Bid/ask Parquet

In [8]:
bid_ask_im_client = icdc.HistoricalPqByCurrencyPairTileClient(
    **config["bid_ask_parquet_config"]
)

In [9]:
bid_ask_data = bid_ask_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
bid_ask_data.head()

,full_symbol,level_1.bid_price.open,level_1.bid_size.open,level_1.ask_price.open,level_1.ask_size.open,level_1.bid_price.close,level_1.bid_size.close,level_1.ask_price.close,level_1.ask_size.close,level_1.bid_price.high,level_1.bid_size.max,level_1.ask_price.high,level_1.ask_size.max,level_1.bid_price.low,level_1.bid_size.min,level_1.ask_price.low,level_1.ask_size.min,level_1.bid_price.mean,level_1.bid_size.mean,level_1.ask_price.mean,level_1.ask_size.mean,knowledge_timestamp
timestamp,,,,,,,,,,,,,,,,,,,,,,
2023-09-11 00:01:00+00:00,binance::APE_USDT,1.2250,51437.00,1.226,25781.00,1.224,17722.00,1.2250,46103.00,1.225,60858.00,1.2260,65356.00,1.2220,6083.00,1.2230,7299.00,1.223341,27860.995192,1.224341,35473.754808,2023-09-19 14:47:21.627940+00:00
2023-09-11 00:01:00+00:00,binance::AVAX_USDT,9.4570,131.00,9.458,930.00,9.454,596.00,9.4550,209.00,9.457,1115.00,9.4580,1552.00,9.4470,6.00,9.4480,2.00,9.452654,520.483871,9.453691,492.668203,2023-09-19 14:47:21.627940+00:00
2023-09-11 00:01:00+00:00,binance::AXS_USDT,4.3890,312.00,4.390,688.00,4.391,1030.00,4.3920,481.00,4.391,1691.00,4.3920,3818.00,4.3840,17.00,4.3850,18.00,4.388183,704.403846,4.389192,760.812500,2023-09-19 14:47:21.627940+00:00
2023-09-11 00:01:00+00:00,binance::BAKE_USDT,0.0949,159855.00,0.095,66417.00,0.095,106674.00,0.0951,233652.00,0.095,193894.00,0.0951,233652.00,0.0948,6585.00,0.0949,2513.00,0.094926,127695.973262,0.095026,172666.540107,2023-09-19 14:47:21.627940+00:00
2023-09-11 00:01:00+00:00,binance::BNB_USDT,212.2700,56.21,212.280,37.82,212.300,5.41,212.3100,47.13,212.390,101.15,212.4000,79.37,212.2700,0.46,212.2800,0.11,212.311076,39.740762,212.321076,41.236323,2023-09-19 14:47:21.627940+00:00


## OHLCV Database

In [10]:
ohlcv_db_im_client = icdcl.CcxtSqlRealTimeImClient(**config["ohlcv_db_config"])

/app/amp/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [11]:
ohlcv_db_data = ohlcv_db_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
ohlcv_db_data.head()

,knowledge_timestamp,open,high,low,close,volume,end_download_timestamp,id,full_symbol
timestamp,,,,,,,,,
2023-09-11 00:00:00+00:00,2023-09-11 00:00:10.176577+00:00,1.2250,1.226,1.2240,1.2250,21814.00,2023-09-11 00:00:10.149844+00:00,237185526,binance::APE_USDT
2023-09-11 00:00:00+00:00,2023-09-11 00:00:10.176577+00:00,9.4570,9.460,9.4570,9.4570,1505.00,2023-09-11 00:00:10.148685+00:00,237185413,binance::AVAX_USDT
2023-09-11 00:00:00+00:00,2023-09-11 00:00:10.176577+00:00,4.3890,4.390,4.3890,4.3890,1179.00,2023-09-11 00:00:10.158478+00:00,237186251,binance::AXS_USDT
2023-09-11 00:00:00+00:00,2023-09-11 00:00:10.176577+00:00,0.0949,0.095,0.0948,0.0949,135342.00,2023-09-11 00:00:10.155999+00:00,237186030,binance::BAKE_USDT
2023-09-11 00:00:00+00:00,2023-09-11 00:00:10.176577+00:00,212.2500,212.290,212.2500,212.2800,120.72,2023-09-11 00:00:10.149208+00:00,237185470,binance::BNB_USDT


# Mock `ImClient`

In [12]:
df = cofinanc.get_MarketData_df6(full_symbols)
df.head()

,full_symbol,open,high,low,close,volume,feature1
timestamp,,,,,,,
2000-01-01 14:31:00+00:00,binance::APE_USDT,100,101,99,101.0,0,1.0
2000-01-01 14:31:00+00:00,binance::AVAX_USDT,100,101,99,101.0,0,1.0
2000-01-01 14:31:00+00:00,binance::AXS_USDT,100,101,99,101.0,0,1.0
2000-01-01 14:31:00+00:00,binance::BAKE_USDT,100,101,99,101.0,0,1.0
2000-01-01 14:31:00+00:00,binance::BNB_USDT,100,101,99,101.0,0,1.0


In [13]:
dataframe_im_client = icdc.DataFrameImClient(
    df,
    full_symbols,
)

In [14]:
start_timestamp = pd.Timestamp("2000-01-01 09:35:00-05:00")
end_timestamp = pd.Timestamp("2000-01-01 10:31:00-05:00")
#
ohlcv_from_df_data = dataframe_im_client.read_data(
    full_symbols,
    start_timestamp,
    end_timestamp,
    config["columns"],
    config["filter_data_mode"],
)
ohlcv_from_df_data.head()

,full_symbol,open,high,low,close,volume,feature1
timestamp,,,,,,,
2000-01-01 14:35:00+00:00,binance::APE_USDT,100,101,99,101.0,4,1.0
2000-01-01 14:35:00+00:00,binance::AVAX_USDT,100,101,99,101.0,4,1.0
2000-01-01 14:35:00+00:00,binance::AXS_USDT,100,101,99,101.0,4,1.0
2000-01-01 14:35:00+00:00,binance::BAKE_USDT,100,101,99,101.0,4,1.0
2000-01-01 14:35:00+00:00,binance::BNB_USDT,100,101,99,101.0,4,1.0
